<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/gpt_4_1_baseline%2Bfewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# =========================
# 0. CLIENT
# =========================

client = OpenAI(api_key="")

# =========================
# 1. MODÈLE
# =========================

MODEL_NAME = "gpt-4.1"

# =========================
# 2. FEW-SHOT EXAMPLES
# =========================

FEW_SHOT_BLOCK = """
Exemples annotés :

Expression : lune de miel
Contexte : Ils sont partis en lune de miel après le mariage.
Catégorie : Expression_idiomatique
Explication : Le sens ne correspond pas à la somme littérale des mots.

Expression : fil rouge
Contexte : Ce thème sert de fil rouge au roman.
Catégorie : Collocation_opaque
Explication : L’expression repose sur une métaphore.

Expression : roman policier
Contexte : Elle lit un roman policier.
Catégorie : Collocation_transparente
Explication : Le sens est déductible des mots.

Expression : livre intéressant
Contexte : C’est un livre intéressant.
Catégorie : Expression_libre
Explication : Combinaison libre sans figement particulier.

---
"""

# =========================
# 3. BASELINE PROMPT
# =========================

BASELINE_PROMPT = """
Tu es linguiste.

Ta tâche est de classer une expression nominale française
dans UNE SEULE des catégories suivantes.

Catégories :

1) Expression_idiomatique
Expression figée dont le sens global n’est pas directement déductible
du sens littéral de ses mots.

2) Collocation_opaque
Association impliquant une métaphore ou une métonymie.

3) Collocation_transparente
Association dont le sens est déductible.

4) Expression_libre
Combinaison construite librement en discours.

Contraintes :
- Choisis UNE seule catégorie.
- Réponds uniquement selon le format ci-dessous.

FORMAT OBLIGATOIRE :
Catégorie : <Expression_idiomatique | Collocation_opaque | Collocation_transparente | Expression_libre>
Explication : <une phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ALLOWED = {
    "Expression_idiomatique",
    "Collocation_opaque",
    "Collocation_transparente",
    "Expression_libre",
}

# =========================
# 4. APPEL LLM (few-shot)
# =========================

def classify_expression_fewshot(expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = (
        FEW_SHOT_BLOCK
        + BASELINE_PROMPT.format(
            expression=str(expression).strip(),
            contexte=contexte
        )
    )

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=100,
    )

    text = (resp.choices[0].message.content or "").strip()

    # validation minimale du format
    cat = None
    for line in text.splitlines():
        if line.lower().startswith("catégorie"):
            cat = line.split(":", 1)[-1].strip()
            break

    if cat not in ALLOWED:
        text = (
            "Catégorie : INVALID\n"
            "Explication : Réponse hors format.\n\n"
            + text
        )

    return text

# =========================
# 5. TRAITEMENT EXCEL
# =========================

input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Baseline GPT-4.1 few-shot"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    df.at[idx, "llm_raw_response"] = classify_expression_fewshot(expr, ex)

output_file = "annotated_nominal_part_7_gpt41_baseline_fewshot.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")
